# Segmenting and Clustering Neighborhoods in Toronto

### Importing python libraries and dependencies

In [1]:
!pip install bs4
!pip install folium
!pip install html5lib

     |████████████████████████████████| 115 kB 4.2 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=b9c04d2844d68d42bf13793ec7431797333f5f2d4be83acaebedf04833a064a1
  Stored in directory: /home/jovyan/.cache/pip/wheels/19/f5/6d/a97dd4f22376d4472d5f4c76c7646876052ff3166b3cf71050
Successfully built bs4
     |████████████████████████████████| 94 kB 2.3 MB/s eta 0:00:011
     |████████████████████████████████| 112 kB 4.0 MB/s eta 0:00:01


In [2]:
import pandas as pd
import requests
import numpy as np
import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

print("All Required Libraries Imported!")

All Required Libraries Imported!


### Webscrapping and data extraction

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data  = requests.get(url).text
soup = BeautifulSoup(data,'html.parser')

In [4]:
#find all html tables in the web page
tables = soup.find('table')

In [5]:
postal_codes = []
boros = []
neiros = []
titles = []

In [6]:
for tr in tables.find_all('tr'):
    
    for td in tr.find_all('td'):
        i=0      
        titles = []
        
        for b in td.find_all('b'):
            postal_code = td.b.text
            postal_codes.append(postal_code)
            
        for a in td.find_all('a'):
            title = a['title']
            titles.append(title)
            i=i+1
                
        if i == 0:
            boro = np.nan
            boros.append(boro)
            neiro = boro
            neiros.append(neiro)
        
        elif i == 1:
            boro = td.a.text
            boros.append(boro)
            neiro = boro
            neiros.append(neiro)
        
        elif i >= 2:
            boro = td.a.text
            boros.append(boro)
            titles.pop(0)
            neiro = titles
            neiros.append(neiro)

### Creating dataframe

In [7]:
#Converting content of PostalCode HTML table as dataframe
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

In [8]:
df['Postalcode'] = postal_codes
df['Borough'] = boros
df['Neighborhood'] = neiros

### Cleaning dataframe

In [9]:
df = df.dropna().reset_index(drop=True)

In [33]:
df.head(50)

,Postalcode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront, Toronto]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,[Islington Avenue]
6,M1B,Scarborough,"[Malvern, Toronto, Rouge, Toronto]"
7,M3B,North York,[Don Mills]
8,M4B,East York,"[Parkview Hill, Woodbine Gardens]"
9,M5B,Downtown Toronto,"[Garden District, Toronto, Ryerson University]"


### Dataframe shape

In [47]:
df.shape

(101, 3)

### Upload CSV

In [20]:
geo_loc = pd.read_csv('Geospatial_Coordinates.csv')

### Merging the dataframes

In [31]:
final_df = pd.merge(df, geo_loc, how='inner', on='Postalcode' )

In [32]:
final_df.head(50)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,[Parkwoods],43.753259,-79.329656
1,M4A,North York,[Victoria Village],43.725882,-79.315572
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront, Toronto]",43.654260,-79.360636
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,[Islington Avenue],43.667856,-79.532242
6,M1B,Scarborough,"[Malvern, Toronto, Rouge, Toronto]",43.806686,-79.194353
7,M3B,North York,[Don Mills],43.745906,-79.352188
8,M4B,East York,"[Parkview Hill, Woodbine Gardens]",43.706397,-79.309937
9,M5B,Downtown Toronto,"[Garden District, Toronto, Ryerson University]",43.657162,-79.378937
